# Análise de Ameaças STRIDE com Azure OpenAI

Este notebook demonstra a lógica principal para analisar um diagrama de arquitetura usando um modelo de visão do Azure OpenAI e a metodologia STRIDE.

**Objetivo:** Enviar uma imagem para a IA e receber um relatório de ameaças de cibersegurança.

### 1. Instalação das Bibliotecas

In [ ]:
!pip install openai python-dotenv

### 2. Configuração das Credenciais do Azure OpenAI

Para segurança, usaremos a biblioteca `getpass` para solicitar as credenciais sem exibi-las na tela.

In [ ]:
import os
from getpass import getpass
from openai import AzureOpenAI
import base64
from google.colab import files
import IPython.display as display

# Solicita as credenciais de forma segura
api_endpoint = getpass("Digite o seu Endpoint do Azure OpenAI: ")
api_key = getpass("Digite a sua Chave da API do Azure OpenAI: ")
api_deployment = getpass("Digite o Nome da sua Implantação (Deployment): ")
api_version = "2024-02-01"

# Inicializa o cliente do Azure OpenAI
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=api_endpoint
)

print("Cliente OpenAI configurado com sucesso!")

### 3. Prompt Engineering: Definindo a Tarefa para a IA

Este é o passo mais importante. Criamos um prompt detalhado que ensina o modelo sobre a metodologia STRIDE e o instrui sobre como realizar a análise.

In [ ]:
PROMPT_STRIDE = """
Você é um especialista em cibersegurança e modelagem de ameaças. Sua tarefa é analisar o diagrama de arquitetura de aplicação fornecido e gerar uma análise de ameaças detalhada baseada na metodologia STRIDE.

A metodologia STRIDE classifica as ameaças em 6 categorias:

1.  **Spoofing (Falsificação de Identidade):** Ameaças que envolvem um ator malicioso se passando por outra entidade.
    * *Pergunta a se fazer:* Quem pode se passar por um usuário, serviço ou componente legítimo?

2.  **Tampering (Violação/Adulteração de Dados):** Ameaças que envolvem a modificação não autorizada de dados.
    * *Pergunta a se fazer:* Onde os dados podem ser modificados sem autorização?

3.  **Repudiation (Repúdio):** Ameaças que permitem que um ator negue ter realizado uma ação por falta de provas.
    * *Pergunta a se fazer:* Como um usuário poderia negar ter feito uma operação? Faltam logs?

4.  **Information Disclosure (Divulgação de Informações):** Ameaças que envolvem a exposição de informações sensíveis.
    * *Pergunta a se fazer:* Onde informações sensíveis podem ser expostas?

5.  **Denial of Service (Negação de Serviço - DoS):** Ameaças que visam tornar um sistema indisponível.
    * *Pergunta a se fazer:* Como um atacante poderia travar ou tornar este sistema indisponível?

6.  **Elevation of Privilege (Elevação de Privilégio):** Ameaças que permitem que um usuário ganhe acesso a funções com privilégios mais altos.
    * *Pergunta a se fazer:* Como um usuário com poucos privilégios poderia ganhar mais acesso do que deveria?

**Sua Tarefa:**
Analise a imagem do diagrama de arquitetura anexa. Para cada componente e fluxo de dados, identifique potenciais ameaças para cada uma das 6 categorias STRIDE. Apresente sua análise em formato Markdown, organizada por categoria.
"""

### 4. Função de Análise e Execução

Agora, criamos uma função para enviar a imagem e o prompt para a OpenAI. Depois, fazemos o upload de uma imagem e executamos a análise.

In [ ]:
def analisar_imagem_stride(image_path):
    """
    Lê uma imagem local, converte para base64 e envia para a análise.
    """
    # Lê a imagem e a codifica em base64
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')

    try:
        print("Enviando imagem para análise... Por favor, aguarde.")
        response = client.chat.completions.create(
            model=api_deployment,
            messages=[
                { "role": "system", "content": "Você é um especialista em cibersegurança." },
                { "role": "user", "content": [
                    { "type": "text", "text": PROMPT_STRIDE },
                    { "type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"} }
                ]}
            ],
            max_tokens=2048
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Ocorreu um erro: {str(e)}"

# --- Execução ---
# 1. Fazer o upload do diagrama de arquitetura
uploaded = files.upload()

# 2. Pegar o nome do arquivo e executar a análise
if uploaded:
    file_name = next(iter(uploaded))
    print(f"\nAnalisando o arquivo: {file_name}")
    
    # Exibe a imagem que foi enviada
    display.display(display.Image(file_name))
    
    # Chama a função de análise
    threat_analysis_report = analisar_imagem_stride(file_name)
    
    # 3. Exibir o resultado formatado em Markdown
    print("\n--- RELATÓRIO DE ANÁLISE DE AMEAÇAS STRIDE ---")
    display.display(display.Markdown(threat_analysis_report))
else:
    print("Nenhum arquivo foi enviado.")